In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install torch

In [ ]:
!pip install torch_geometric

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv
from torch_geometric.utils import add_self_loops
from sklearn.model_selection import train_test_split

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [ ]:
import pandas as pd

# Replace 'your_file.csv' with the name of the file you uploaded
df = pd.read_csv('Lipophilicity!.csv')
df.head()

In [ ]:
# Remove row 1562 (remember, index in Python starts from 0)
df = df.drop(1561, axis=0)

# Reset index if you want to reindex the DataFrame after removal
df = df.reset_index(drop=True)

# Display the updated DataFrame
df.head()

In [ ]:
y=list(df['exp'])
y

In [ ]:
# Convert to the desired format
formatted_values = [[[y_t]] for y_t in y]
# Print the formatted list
len(formatted_values)
y=formatted_values

In [ ]:
SMILES=list(df['smiles'])

In [ ]:
smiles_list=SMILES_rt=SMILES
SMILES_list=SMILES

In [ ]:
!pip install rdkit

Features

In [ ]:
!pip install rdkit

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdMolHash
import rdkit
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem import Draw
from rdkit.Chem.rdMolDescriptors import CalcMolFormula

In [ ]:
angles = pd.read_csv('angles.csv', header=None)
angles = list(angles.to_numpy().flatten())
print(angles)

In [ ]:
dipole_momentum = pd.read_csv('dipole_momentum.csv', header=None)
dipole_momentum = list(dipole_momentum.to_numpy().flatten())
print(dipole_momentum)

In [ ]:
widths = pd.read_csv('widths.csv', header=None)
widths = list(widths.to_numpy().flatten())
print(widths)

In [ ]:
lengths = pd.read_csv('lengths.csv', header=None)
lengths = list(lengths.to_numpy().flatten())
print(lengths)

In [ ]:
heights = pd.read_csv('heights.csv', header=None)
heights = list(heights.to_numpy().flatten())
print(heights)

In [ ]:
volumes = pd.read_csv('volumes.csv', header=None)
volumes = list(volumes.to_numpy().flatten())
print(volumes)

In [ ]:
from rdkit import Chem

# List of SMILES strings
smiles_list = SMILES_rt
# Initialize list to store all edge indexes
all_edge_indexes = []

# Iterate through each SMILES string
for smiles in smiles_list:
    # Convert SMILES string to RDKit Mol object
    mol = Chem.MolFromSmiles(smiles)

    # Get atom and bond information
    atoms = mol.GetAtoms()
    bonds = mol.GetBonds()

    # Initialize edge indexes list for this SMILES
    edge_indexes = []

    # Iterate through bonds to get the edge indexes
    for bond in bonds:
        begin_idx = bond.GetBeginAtomIdx()
        end_idx = bond.GetEndAtomIdx()
        edge_indexes.append([begin_idx, end_idx])
        edge_indexes.append([end_idx, begin_idx])  # Add the reverse direction as well

    # Append the edge indexes for this SMILES to the overall list
    all_edge_indexes.append(edge_indexes)

edge_index=all_edge_indexes
#edge_index

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

# A dictionary mapping atomic numbers to electronegativity values (values taken from Pauling scale)
electronegativity_dict = {
    1: 2.20,   # Hydrogen
    6: 2.55,   # Carbon
    7: 3.04,   # Nitrogen
    8: 3.44,   # Oxygen
    9: 3.98,   # Fluorine
    16: 2.58,  # Sulfur
    17: 2.96,  # Chlorine
    35: 2.66,  # Bromine
    53: 2.66,  # Iodine
    11: 0.93,  # Sodium
    12: 1.31,  # Magnesium
    # Add more elements as needed for your use case
}

x = []
di = 0

while di < len(SMILES_list):
    # Define the SMILES notation of the molecule
    smiles = SMILES_list[di]

    # Convert SMILES to an RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)

    # Initialize an empty list to store element information
    result = []

    # Calculate global features for the molecule
    min_degree = min(atom.GetDegree() for atom in mol.GetAtoms())  # Minimum degree of atoms in the molecule
    num_hbond_donors = sum(1 for atom in mol.GetAtoms() if atom.GetSymbol() == 'O' and atom.GetTotalNumHs() > 0)  # Number of hydrogen bond donors
    num_rings = len(mol.GetRingInfo().AtomRings())  # Number of rings in the molecule
    num_rotatable_bonds = rdMolDescriptors.CalcNumRotatableBonds(mol)  # Number of rotatable bonds
    polar_surface_area = rdMolDescriptors.CalcTPSA(mol)  # Topological Polar Surface Area

    # Additional chemical properties from RDKit
    molecular_weight = rdMolDescriptors.CalcExactMolWt(mol)  # Molecular weight
    logP = rdMolDescriptors.CalcCrippenDescriptors(mol)[0]  # LogP (octanol-water partition coefficient)
    # refractivity = rdMolDescriptors.CalcRefractivity(mol)  # Refractivity

    # Additional properties for the molecule
    num_atoms = mol.GetNumAtoms()  # Number of atoms in the molecule
    # num_heavy_atoms = mol.CountAtoms() - mol.GetNumAtoms()  # Number of non-hydrogen atoms
    hba = rdMolDescriptors.CalcNumHBA(mol)  # Number of hydrogen bond acceptors
    hbd = rdMolDescriptors.CalcNumHBD(mol)  # Number of hydrogen bond donors
    fraction_sp2 = sum(1 for atom in mol.GetAtoms() if atom.GetHybridization() == Chem.HybridizationType.SP2) / num_atoms  # Fraction of SP2 atoms

    # Calculate global properties related to atom's characteristics
    is_in_ring = int(len(mol.GetRingInfo().AtomRings()) > 0)  # Global ring feature: Does the molecule contain any rings?
    is_aromatic = int(any(atom.GetIsAromatic() for atom in mol.GetAtoms()))  # Global aromaticity: Is the molecule aromatic?
    formal_charge = sum(atom.GetFormalCharge() for atom in mol.GetAtoms())  # Global formal charge of the molecule
    valence = sum(atom.GetTotalValence() for atom in mol.GetAtoms())  # Global valence: Sum of all atom valences in the molecule

    # General electronegativity: Calculate the average electronegativity of all atoms
    total_electronegativity = sum(electronegativity_dict.get(atom.GetAtomicNum(), 0) for atom in mol.GetAtoms())
    general_electronegativity = total_electronegativity / num_atoms if num_atoms > 0 else 0  # Average electronegativity

    # Now, let's create a list of global features that will be added to each atom's feature set
    global_features = [
        min_degree,             # Minimum degree of atoms in the molecule
        num_hbond_donors,      # Number of hydrogen bond donors
        num_rings,             # Number of rings in the molecule
        num_rotatable_bonds,   # Number of rotatable bonds
        polar_surface_area,    # Polar surface area
        molecular_weight,      # Molecular weight
        #logP,                  # LogP (octanol-water partition coefficient)
        # refractivity,        # Refractivity
        num_atoms,             # Number of atoms
        hba,                   # Hydrogen bond acceptors
        hbd,                   # Hydrogen bond donors
        fraction_sp2,          # Fraction of SP2 hybridized atoms
        valence,               # Global valence (sum of all atom valences)
        general_electronegativity,  # General electronegativity (average of all atoms' electronegativity)
    ]

    # Iterate through each atom in the molecule
    for atom in mol.GetAtoms():
        element = atom.GetSymbol()

        # Define features that are related to general chemical activity
        features = [
            is_in_ring,            # Whether the molecule contains any rings
            is_aromatic,           # Whether the molecule is aromatic
            formal_charge,         # Global formal charge of the molecule
            int(atom.GetDegree()), # Atomic degree (number of bonds)
            int(atom.GetAtomicNum()),  # Atomic number
            int(atom.GetTotalNumHs()), # Number of hydrogens
            int(atom.GetTotalValence()), # Atomic valence
            int(atom.GetNumRadicalElectrons()),  # Number of radical electrons
            int(atom.GetFormalCharge()), # Formal charge on the atom
            int(atom.GetHybridization()), # Hybridization of the atom
            int((atom.GetMass() - 10.812) / 116.092),  # Normalized mass feature
            int((atom.GetAtomicNum() - 1.5) / 0.6),
            int((atom.GetAtomicNum() - 0.64) / 0.76),
            electronegativity_dict.get(atom.GetAtomicNum(), 0),  # Electronegativity (using a dictionary)
            int(atom.GetAtomicNum() in electronegativity_dict),  # Whether the atom has defined electronegativity
            int(atom.GetNumImplicitHs() > 0),  # If the atom has implicit hydrogens
        ]

        # Add global features (these are the same for every atom in the molecule)
        features.extend(global_features)

        # You can add more features related to the chemical reactivity or functionality of the atom
        # For example, if the atom is part of a common functional group (like hydroxyl group):
        if atom.GetSymbol() == 'O' and any(neighbor.GetSymbol() == 'H' for neighbor in atom.GetNeighbors()):
            features.append(1)  # Hydroxyl group (-OH) presence
        else:
            features.append(0)

        # Add additional features from other lists (volumes, widths, lengths, etc.)
        for additional_list in [volumes, widths, lengths, heights, dipole_momentum, angles]:
            additional_feature = additional_list[di]  # Assuming these lists are defined and aligned with SMILES_list
            features.append(additional_feature)

        # Append the element information to the list
        result.append(features)

    # Append the result for the current molecule to x
    x.append(result)
    di += 1

# Print the final list of features
print("x =", x)

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

# A dictionary mapping atomic numbers to electronegativity values (values taken from Pauling scale)
electronegativity_dict = {
    1: 2.20,   # Hydrogen
    6: 2.55,   # Carbon
    7: 3.04,   # Nitrogen
    8: 3.44,   # Oxygen
    9: 3.98,   # Fluorine
    16: 2.58,  # Sulfur
    17: 2.96,  # Chlorine
    35: 2.66,  # Bromine
    53: 2.66,  # Iodine
    11: 0.93,  # Sodium
    12: 1.31,  # Magnesium
    # Add more elements as needed for your use case
}

x = []
di = 0

while di < len(SMILES_list):
    # Define the SMILES notation of the molecule
    smiles = SMILES_list[di]

    # Convert SMILES to an RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)

    # Initialize an empty list to store element information
    result = []

    # Calculate global features for the molecule
    min_degree = min(atom.GetDegree() for atom in mol.GetAtoms())  # Minimum degree of atoms in the molecule
    num_hbond_donors = sum(1 for atom in mol.GetAtoms() if atom.GetSymbol() == 'O' and atom.GetTotalNumHs() > 0)  # Number of hydrogen bond donors
    num_rings = len(mol.GetRingInfo().AtomRings())  # Number of rings in the molecule
    num_rotatable_bonds = rdMolDescriptors.CalcNumRotatableBonds(mol)  # Number of rotatable bonds
    polar_surface_area = rdMolDescriptors.CalcTPSA(mol)  # Topological Polar Surface Area

    # Additional chemical properties from RDKit
    molecular_weight = rdMolDescriptors.CalcExactMolWt(mol)  # Molecular weight
    logP = rdMolDescriptors.CalcCrippenDescriptors(mol)[0]  # LogP (octanol-water partition coefficient)
    # refractivity = rdMolDescriptors.CalcRefractivity(mol)  # Refractivity

    # Additional properties for the molecule
    num_atoms = mol.GetNumAtoms()  # Number of atoms in the molecule
    # num_heavy_atoms = mol.CountAtoms() - mol.GetNumAtoms()  # Number of non-hydrogen atoms
    hba = rdMolDescriptors.CalcNumHBA(mol)  # Number of hydrogen bond acceptors
    hbd = rdMolDescriptors.CalcNumHBD(mol)  # Number of hydrogen bond donors
    fraction_sp2 = sum(1 for atom in mol.GetAtoms() if atom.GetHybridization() == Chem.HybridizationType.SP2) / num_atoms  # Fraction of SP2 atoms

    # Calculate global properties related to atom's characteristics
    is_in_ring = int(len(mol.GetRingInfo().AtomRings()) > 0)  # Global ring feature: Does the molecule contain any rings?
    is_aromatic = int(any(atom.GetIsAromatic() for atom in mol.GetAtoms()))  # Global aromaticity: Is the molecule aromatic?
    formal_charge = sum(atom.GetFormalCharge() for atom in mol.GetAtoms())  # Global formal charge of the molecule
    valence = sum(atom.GetTotalValence() for atom in mol.GetAtoms())  # Global valence: Sum of all atom valences in the molecule

    # General electronegativity: Calculate the average electronegativity of all atoms
    total_electronegativity = sum(electronegativity_dict.get(atom.GetAtomicNum(), 0) for atom in mol.GetAtoms())
    general_electronegativity = total_electronegativity / num_atoms if num_atoms > 0 else 0  # Average electronegativity

    # Now, let's create a list of global features that will be added to each atom's feature set
    global_features = [
        min_degree,             # Minimum degree of atoms in the molecule
        num_hbond_donors,      # Number of hydrogen bond donors
        num_rings,             # Number of rings in the molecule
        num_rotatable_bonds,   # Number of rotatable bonds
        polar_surface_area,    # Polar surface area
        molecular_weight,      # Molecular weight
        #logP,                  # LogP (octanol-water partition coefficient)
        # refractivity,        # Refractivity
        num_atoms,             # Number of atoms
        hba,                   # Hydrogen bond acceptors
        hbd,                   # Hydrogen bond donors
        fraction_sp2,          # Fraction of SP2 hybridized atoms
        valence,               # Global valence (sum of all atom valences)
        general_electronegativity,  # General electronegativity (average of all atoms' electronegativity)
    ]

    # Iterate through each atom in the molecule
    for atom in mol.GetAtoms():
        element = atom.GetSymbol()

        # Define features that are related to general chemical activity
        features = [
            is_in_ring,            # Whether the molecule contains any rings
            is_aromatic,           # Whether the molecule is aromatic
            formal_charge,         # Global formal charge of the molecule
            int(atom.GetDegree()), # Atomic degree (number of bonds)
            int(atom.GetAtomicNum()),  # Atomic number
            int(atom.GetTotalNumHs()), # Number of hydrogens
            int(atom.GetTotalValence()), # Atomic valence
            int(atom.GetNumRadicalElectrons()),  # Number of radical electrons
            int(atom.GetFormalCharge()), # Formal charge on the atom
            int(atom.GetHybridization()), # Hybridization of the atom
            int((atom.GetMass() - 10.812) / 116.092),  # Normalized mass feature
            int((atom.GetAtomicNum() - 1.5) / 0.6),
            int((atom.GetAtomicNum() - 0.64) / 0.76),
            electronegativity_dict.get(atom.GetAtomicNum(), 0),  # Electronegativity (using a dictionary)
            int(atom.GetAtomicNum() in electronegativity_dict),  # Whether the atom has defined electronegativity
            int(atom.GetNumImplicitHs() > 0),  # If the atom has implicit hydrogens
        ]

        # Add global features (these are the same for every atom in the molecule)
        features.extend(global_features)

        # You can add more features related to the chemical reactivity or functionality of the atom
        # For example, if the atom is part of a common functional group (like hydroxyl group):
        if atom.GetSymbol() == 'O' and any(neighbor.GetSymbol() == 'H' for neighbor in atom.GetNeighbors()):
            features.append(1)  # Hydroxyl group (-OH) presence
        else:
            features.append(0)

        # Add additional features from other lists (volumes, widths, lengths, etc.)
        for additional_list in [volumes, widths, lengths, heights, dipole_momentum, angles]:
            additional_feature = additional_list[di]  # Assuming these lists are defined and aligned with SMILES_list
            features.append(additional_feature)

        # Append the element information to the list
        result.append(features)

    # Append the result for the current molecule to x
    x.append(result)
    di += 1

# Print the final list of features
print("x =", x)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.utils import add_self_loops
import random
import csv
import os

# Create directory for logs if it doesn't exist
os.makedirs('logs', exist_ok=True)

# Define your GSR model
class GSR(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GSR, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.conv4 = GATConv(hidden_channels, hidden_channels)
        self.conv5 = GATConv(hidden_channels, hidden_channels)
        self.conv3 = GATConv(hidden_channels, out_channels)  # Output layer

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Apply the graph convolutional layers
        x = self.conv1(x, edge_index)
        x = torch.tanh(x)

        x = self.conv2(x, edge_index)
        x = torch.tanh(x)

        x = self.conv4(x, edge_index)
        x = torch.tanh(x)

        x = self.conv5(x, edge_index)
        x = torch.tanh(x)

        # Final output layer
        x = self.conv3(x, edge_index)

        return x

# Assuming `x`, `edge_index`, and `y` are the data variables provided
# Convert data into Data objects
data_list = []
for i in range(len(x)):
    x_graph = torch.tensor(x[i], dtype=torch.float32)
    edge_index_graph = torch.tensor(edge_index[i], dtype=torch.long).t().contiguous()
    y_graph = torch.tensor(y[i], dtype=torch.float32)

    data_graph = Data(x=x_graph, edge_index=edge_index_graph, y=y_graph)
    data_list.append(data_graph)

train_data = data_list[:int(0.8*len(data_list))]
val_data = data_list[int(0.8*len(data_list)):int(0.9*len(data_list))]
test_data = data_list[int(0.9*len(data_list)):]

# No batching, so DataLoader is not used here
train_loader = train_data
val_loader = val_data
test_loader = test_data

# Initialize the GSR model
model = GSR(in_channels=35, hidden_channels=28, out_channels=1)
optimizer = optim.RMSprop(model.parameters(), lr=0.00075)
criterion = nn.MSELoss()

# Function to compute RMSE
def compute_rmse(predictions, targets):
    # Calculate the RMSE by taking the square root of MSE
    mse = ((predictions - targets) ** 2).mean()
    rmse = torch.sqrt(mse)
    return rmse

# Setup CSV files for logging
metrics_file = open('logs/metrics.csv', 'w', newline='')
metrics_writer = csv.writer(metrics_file)
metrics_writer.writerow(['Epoch', 'Train_Loss', 'Train_RMSE', 'Val_Loss', 'Val_RMSE', 'Test_Loss', 'Test_RMSE'])

# Training loop
for epoch in range(1250):
    model.train()
    total_loss = 0
    total_train_rmse = 0


    # Training on individual graphs
    for idx, data_graph in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data_graph)
        loss = criterion(output, data_graph.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Calculate RMSE for each batch in the training set
        train_rmse = compute_rmse(output, data_graph.y)
        total_train_rmse += train_rmse.item()

    # Evaluate on the validation and test data every 50 epochs
    if epoch % 50 == 0:
        model.eval()
        val_loss = 0
        total_val_rmse = 0
        
        # Create epoch-specific CSV files for y_test and y_pred
        y_test_file = open(f'logs/y_test_epoch_{epoch}.csv', 'w', newline='')
        y_pred_file = open(f'logs/y_pred_epoch_{epoch}.csv', 'w', newline='')
        y_test_writer = csv.writer(y_test_file)
        y_pred_writer = csv.writer(y_pred_file)
        
        # Write headers
        y_test_writer.writerow(['Sample_ID', 'y_test'])
        y_pred_writer.writerow(['Sample_ID', 'y_pred'])
        
        with torch.no_grad():
            # Validation
            for i, data_graph in enumerate(val_loader):
                y_pred = model(data_graph)
                batch_val_loss = criterion(y_pred, data_graph.y)
                val_loss += batch_val_loss.item()

                # Calculate RMSE for validation
                val_rmse = compute_rmse(y_pred, data_graph.y)
                total_val_rmse += val_rmse.item()

                # Print RMSE and loss for each element in the validation set
                #print(f"element {i}, Epoch {epoch}: Validation Loss: {batch_val_loss.item():.4f}")
                #print(f"element {i}, Epoch {epoch}: Validation RMSE: {val_rmse.item():.4f}")
                #print(f'y_val: {data_graph.y.tolist()}, y_pred: {y_pred.tolist()}')

            # Testing
            test_loss = 0
            total_test_rmse = 0
            for i, data_graph in enumerate(test_loader):
                y_pred = model(data_graph)
                batch_test_loss = criterion(y_pred, data_graph.y)
                test_loss += batch_test_loss.item()

                # Calculate RMSE for test
                test_rmse = compute_rmse(y_pred, data_graph.y)
                total_test_rmse += test_rmse.item()

                # Print RMSE and loss for each element in the test set
                print(f"element {i}, Epoch {epoch}: Test Loss: {batch_test_loss.item():.4f}")
                print(f"element {i}, Epoch {epoch}: Test RMSE: {test_rmse.item():.4f}")
                print(f'y_test: {data_graph.y.tolist()}, y_pred: {y_pred.tolist()}')
                
                # Write y_test and y_pred values to respective CSV files
                for j, (y_t, y_p) in enumerate(zip(data_graph.y.tolist(), y_pred.tolist())):
                    sample_id = f'test_{i}_{j}'
                    y_test_writer.writerow([sample_id, y_t])
                    y_pred_writer.writerow([sample_id, y_p])

        # Close the y_test and y_pred files
        y_test_file.close()
        y_pred_file.close()
        
        # Calculate averages
        avg_train_loss = total_loss/len(train_loader)
        avg_train_rmse = total_train_rmse/len(train_loader)
        avg_val_loss = val_loss/len(val_loader)
        avg_val_rmse = total_val_rmse/len(val_loader)
        avg_test_loss = test_loss/len(test_loader)
        avg_test_rmse = total_test_rmse/len(test_loader)
        # Write metrics to CSV
        metrics_writer.writerow([epoch, avg_train_loss, avg_train_rmse, 
                                avg_val_loss, avg_val_rmse, 
                                avg_test_loss, avg_test_rmse])
        metrics_file.flush()  # Ensure data is written to file

        # Print average RMSE and loss every 50 epochs
        #print(f"Epoch {epoch}, Average Training Loss: {avg_train_loss:.4f}")
        #print(f"Epoch {epoch}, Average Training RMSE: {avg_train_rmse:.4f}")
        #print(f"Epoch {epoch}, Average Validation Loss: {avg_val_loss:.4f}")
        #print(f"Epoch {epoch}, Average Validation RMSE: {avg_val_rmse:.4f}")
        #print(f"Epoch {epoch}, Average Test Loss: {avg_test_loss:.4f}")
        #print(f"Epoch {epoch}, Average Test RMSE: {avg_test_rmse:.4f}")
    print(epoch, total_loss, total_train_rmse)

# Close the metrics file
metrics_file.close()